In [1]:
import os
import pandas as pd
import numpy as np
import math
from ipywidgets import interactive, IntSlider, FloatSlider

In [2]:
from py.helper import AOI
from py.helper import Intersection
from py.helper import translate, rotate_x, rotate_y, rotate_z, scale_x, scale_y, scale_z
from py.helper import plot, plot_with_image, plot_vectors, plot_vectors_with_image, plot_vectors_with_image_and_aois, plot_heatmap
from py.helper import apply_transformation, normalize, get_all_aois_intersection, set_closest_intersection_and_get_distance, get_gaze_pairs

In [3]:
def get_aois():
    schrank_oben = AOI([0, 330, 75], [800, 330, 75], [0, 496, 75], 'orange', 'Schrank oben')
    wand = AOI([0, 330, 1], [0, 200, 1], [800, 330, 1], 'blue', 'Wand zwischen Schränken')
    schrank_unten = AOI([0, 200, 156], [800, 200, 156], [0, 0, 156], 'white', 'Schrank unten')
    af = AOI([0, 200, 0], [800, 200, 0], [0, 200, 156], 'green', 'Arbeitsfläche')
    ks = AOI([800, 0, 156], [935, 0, 156], [800, 496, 156], 'cyan', 'Kühlschrank')
    
    return [schrank_oben, wand, schrank_unten, ks, af]

In [4]:
def run_aoi_evaluation(aois, transformation_matrix, coordinates, gazes, heatmap_prefix):
    # normierte Blickrichtung auf Startkoordinaten addieren
    gaze_ends = np.swapaxes(np.swapaxes(coordinates, 0, 1) + np.swapaxes(gazes, 0, 1) * 10000, 0, 1)

    # transformieren
    transformed_coordinates = apply_transformation(coordinates, transformation_matrix, swap=False)
    transformed_gaze_ends = apply_transformation(gaze_ends, transformation_matrix, swap=False)

    # Blickrichtung zurückrechnen und normalisieren
    transformed_directions = transformed_gaze_ends - transformed_coordinates
    transformed_gazes = np.array([normalize(transformed_directions[i]) for i in range(len(transformed_directions))])

    # Intersektion Ojekte erstellen
    intersections = get_all_aois_intersection(transformed_coordinates, transformed_gazes, aois)

    # Kürzeste Entfernung für jeden Punkt
    distances = np.array([set_closest_intersection_and_get_distance(intersections[i]) for i in range(len(intersections))])

    # Start- und Endpunkte der Blicke (Start = Ende, wenn kein Schnittpunkt => Entfernung INF)
    gaze_starts, gaze_ends = get_gaze_pairs(transformed_coordinates, transformed_gazes, distances)

    gaze_starts = np.swapaxes(gaze_starts, 0, 1)
    gaze_ends = np.swapaxes(gaze_ends, 0, 1)

    #plot_vectors(gaze_starts, gaze_ends, 'Test')
    #plot_vectors_with_image_and_aois(gaze_starts, gaze_ends, aois, 'Test')

    for aoi in aois:
        plot_heatmap(aoi, aoi.title, save=True, filename='%s%s.png' % (heatmap_prefix, aoi.title))

In [5]:
df = pd.read_csv('./2_raw/2019-02-27/webcam_2019-02-27-14-57.csv', sep=', ', engine='python')

gaze_df = df[(df[['success']] == 1).all(axis=1)]
gaze_df = gaze_df[(gaze_df[['gaze_0_x','gaze_0_y','gaze_0_z']] != 0).all(axis=1)]

coords = np.array([gaze_df['eye_lmk_X_0'], gaze_df['eye_lmk_Y_0'], gaze_df['eye_lmk_Z_0']])
gazes = np.array([gaze_df['gaze_0_x'], gaze_df['gaze_0_y'], gaze_df['gaze_0_z']])

In [6]:
def get_transformation_matrix():
    rot_x = rotate_x(math.radians(10))
    rot_y = rotate_y(math.radians(30))
    rot_z = rotate_z(math.radians(-5))

    s_x = scale_x(0.4)
    s_y = scale_y(0.4)
    s_z = scale_z(0.4)

    d = translate(125, 400, 25)
    
    return scale_x(-1) @ rotate_y(math.radians(-90)) @ d @ s_z @ s_y @ s_x @ rot_z @ rot_y @ rot_x @ scale_y(-1)

run_aoi_evaluation(get_aois(), get_transformation_matrix(), coords, gazes, './images/test/')

<Figure size 1440x360 with 0 Axes>

In [7]:
# siehe https://stackoverflow.com/questions/19587118/iterating-through-directories-with-python

root = './0_sorted'
FILE_NAMES = []

for subdir, dirs, files in os.walk(root):
    for file in files:
        if 'webcam_2019' not in file or '.csv' not in file:
            continue
        FILE_NAMES.append(os.path.join(subdir, file).replace('\\', '/'))

In [8]:
for f in FILE_NAMES:
    df = pd.read_csv(f, sep=', ', engine='python')
    
    gaze_df = df[(df[['success']] == 1).all(axis=1)]
    gaze_df = gaze_df[(gaze_df[['gaze_0_x','gaze_0_y','gaze_0_z']] != 0).all(axis=1)]

    coords = np.array([gaze_df['eye_lmk_X_0'], gaze_df['eye_lmk_Y_0'], gaze_df['eye_lmk_Z_0']])
    gazes = np.array([gaze_df['gaze_0_x'], gaze_df['gaze_0_y'], gaze_df['gaze_0_z']])
    
    path = f.replace('./0_sorted', './images').replace('.csv','/')
    
    run_aoi_evaluation(get_aois(), get_transformation_matrix(), coords, gazes, path)
    

<Figure size 1440x360 with 0 Axes>